# Getting Started with Llamaindex and Vercel

First make sure you have a recent version of node

## Create a Llamaindex project
- `npx create-llama@latest`
  - Project Name? llama-vercel
  - Template? Chat
  - Framework? FastAPI (Python)
  - Generate a NextJS frontend? Yes
  - Obervability? No
  - OpenAI key?
    - you can use mine if you want to take this approach
  - Data source? Example PDF
  - Another data source? No
  - LlamaParse? No
  - Vector database? ChromaDB
  - Agent? leave blank and just press enter
  - Proceed? Generate code and install dependencies

### Start the backend
- `cd llama-vercel/backend`
- `poetry install`
- `poetry shell`
- edit .env: uncomment CHROMA_PATH and set it to `chroma`
- `poetry run generate`
- `python main.py`
- read backend/README.md for more information

### Start the frontend
- open a new terminal window
- `cd ../frontend`
- `npm install`
- `npm run dev`
- visit http://localhost:3000

## Create a nextjs-fastapi project and deploy it on Vercel
- Sign into https://vercel.com/ using your github account
- Go to https://vercel.com/templates/next.js/nextjs-fastapi-starter
- Click deploy
    - this creates a new github repo using the nextjs-fastapi-starter template
    - and deploys your new repo to vercel
- Click "Continue to dashboard"
- Click on the thumbnail on the left side of the page
    - That's your new project!

### Run the project locally
- `git clone https://github.com/[username]/nextjs-fastapi-starter`
- `cd nextjs-fastapi-starter`
- `npm install`
- `npm run dev`
- Go to http://localhost:3000

## Deploy Llamaindex project to Vercel

### Initial commit to a new llama-vercel repo on github
- Create a llama-vercel repo on github
- `cd ~/llama-vercel`
- `git add .`
- `git commit -m 'first commit'`
- `git remote add origin https://github.com/[username]/llama-vercel.git`
- `git branch -M main`
- `git push -u origin main`

### Make the project structure compatible with Vercel
- `mv backend api`
- `mv frontend/README.md frontend/README-frontend.md`
- `mv frontend/* .`
- `mv frontend/.* .`
- `rmdir frontend`

### Make api request forwarding compatible with Vercel
- edit `.env` and set NEXT_PUBLIC_CHAT_API=/api/chat
- comment out lines 6-9 in `app/components/ui/lib/url.ts`
- rm `next.config.json`
- replace the contents of `next.config.mjs` with the following:
```
/** @type {import('next').NextConfig} */
import fs from "fs";
import webpack from "./webpack.config.mjs";

const nextConfig = {
  "experimental": {
    "outputFileTracingIncludes": {
      "/*": [
        "./cache/**/*"
      ]
    },
    "serverComponentsExternalPackages": [
      "sharp",
      "onnxruntime-node"
    ]
  },
  "images": {
    "unoptimized": true
  },
  "webpack": webpack,
  "rewrites": async () => {
    return [
      {
        source: "/api/:path*",
        destination:
          process.env.NODE_ENV === "development"
            ? "http://127.0.0.1:8000/api/:path*"
            : "/api/",
      },
    ];
  },
}

export default nextConfig;
```

### Reduce the number of files in the api directory
Vercel creates a serverless function for each file, and you must have fewer than 12 files on the free (Hobby) plan
- `rm api/app/observability.py`
- edit `api/main.py` and remove the init_observability import and function call

### Downgrade to python 3.9 for Vercel
In the api directory
- edit `pyproject.toml` and set the python version to ^3.9
- `poetry lock --no-update`

### Reinstall poetry virtual environment (because we changed backend to api)
In the api directory
- `poetry env remove --all`
- `poetry install`

### Generate a requirements.txt file from pyproject.toml
In the api directory
- if your version of poetry is below 1.8,
    - run `poetry self update` or re-install poetry to get poetry 1.8 or later
- add poetry-plugin-export:
    - run `pip install --user poetry-plugin-export` or `poetry self add poetry-plugin-export`
- create requirements.txt
    - `poetry export -f requirements.txt --without-hashes --output ../requirements.txt`

### Test the project locally

Start the backend (in the api directory)
- `poetry shell`
- `python main.py`

Start the frontend (do this in a new terminal window in the project root directory)
- `npm run dev`
- visit http://localhost:3000

### Push changes to github
In the project root
- `git add .`
- `git commit -m 'prepare to deploy to vercel'`
- `git push`

### Create a llama-vercel project on vercel
- Go to https://vercel.com and sign in as your github user
- Add a new project
- Import your llama-vercel repo
- Change the Framework preset to Next.js
- Set the Node.js version to 18.x
- paste the contents of api/.env to the environment variables section
- click Deploy